In [ ]:
import os
from sentinelsat import SentinelAPI
import pandas as pd
import datetime
from datetime import timedelta
import json
import requests
from datetime import date, timedelta

home = '/Users/stijn/Documents/jaar 2/UC/urbancomputing_group2'
TROPOMI = home + '/data/TROPOMI/'

In [ ]:
# get the access token

def get_access_token(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
        }
    try:
        r = requests.post("https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
        data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Access token creation failed. Reponse from the server was: {r.json()}"
            )
    return r.json()["access_token"]

username = "stijnvleugels@gmail.com"
password = "agk!h7z9AK.zuGM"      
access_token = get_access_token(username, password)

In [ ]:
# find files in the area of interest
data_collection = "SENTINEL-5P"

offset = 0.5 # increase size of area of interest 
lon_min = 19.5 - offset ; lat_min = 31.5 + offset ; lon_max = 29.5 - offset ; lat_max = 34.2 + offset

aoi = f"POLYGON(({lon_min - offset} {lat_min - offset},{lon_max + offset} {lat_min - offset},{lon_max + offset} {lat_max + offset},{lon_min - offset} {lat_max + offset},{lon_min - offset} {lat_min - offset}))'"

start_date = date.fromisoformat('2019-06-01')
end_date = date.fromisoformat('2019-06-02')

while end_date != date.fromisoformat('2019-07-01'): # we can only return 20 results(?) at a time so use this loop
    # it is still missing some images e.g. on 2019-06-30, not sure what's going wrong.

    # get IDs and filenames of L2__NO2____ images in the area of interest
    json = requests.get(f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'NRTI_L2__NO2') and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}) and ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T00:00:00.000Z").json()
    values = pd.DataFrame.from_dict(json['value'])
    ids = values['Id']
    names = values['Name']

    for i,id_im in enumerate(ids): # download the images
        url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({id_im})/$value"

        name = names[i]
        headers = {"Authorization": f"Bearer {access_token}"}

        session = requests.Session()
        session.headers.update(headers)
        response = session.get(url, headers=headers, stream=True)

        if response.headers['Content-Type'] == 'application/zip': # these contain the data
            if response.headers['content-disposition'][30:41] == 'L2__NO2____':
                with open(TROPOMI + f"{name}.zip", "wb") as file:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk: 
                            file.write(chunk)

    # move to next day
    start_date += timedelta(days=1)
    end_date = start_date + timedelta(days=1)

In [ ]:
# unzip all the files
import zipfile
for filename in os.listdir(TROPOMI):
    if filename.endswith(".zip"):
        path_to_zip_file = TROPOMI + filename
        directory_to_extract_to = TROPOMI
        with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
            zip_ref.extractall(directory_to_extract_to)

# 